In [29]:
from pprint import pprint
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .appName("read_parking_data")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [30]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [31]:
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.87:9000/parking-citations.csv')\
    .cache()

In [32]:
data_frame.count()

9881842

In [33]:
#Counting number of partitions in the RDD
data_frame.rdd.getNumPartitions()

11

In [34]:
#Printing the Schema for the DataFrame
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [35]:
#Drop the Columns Agency Description, Agency, and Route
data_frame = data_frame.drop("Agency Description", "Agency","Route")

In [36]:
#Fine amount to float
data_frame = data_frame.withColumn('Fine amount',data_frame['Fine amount'].cast("float"))

In [37]:
from pyspark.sql.functions import col, avg

In [38]:
#printing mean fine amount
data_frame.select(avg(col("Fine amount"))).show()

+----------------+
|avg(Fine amount)|
+----------------+
|70.1855354220642|
+----------------+



In [39]:
import pyspark.sql.functions as parv

In [40]:
#Showing top 10 most frequent vehicle makes, and their frequencies
data_frame1 = data_frame.withColumn('Vehicle_brands', parv.explode(parv.split(parv.col('Make'), ' ')))\
    .groupBy('Vehicle_brands')\
    .count()\
    .sort('count', ascending=False)\
    .show(10)

+--------------+-------+
|Vehicle_brands|  count|
+--------------+-------+
|          TOYT|1633266|
|          HOND|1113834|
|          FORD| 860828|
|          NISS| 709250|
|          CHEV| 674422|
|           BMW| 450909|
|          MERZ| 402126|
|          VOLK| 335618|
|          HYUN| 304934|
|          DODG| 290979|
+--------------+-------+
only showing top 10 rows



In [41]:
#Colors Dictionary
COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White',
'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

In [42]:
#Function for expanding the abbreviation 
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


def abbreviations(color):
    if color in COLORS:
        return COLORS[color]
    else: 
        return color

udf_abbreviations = udf(abbreviations, StringType())
data_frame = data_frame.withColumn("color long",udf_abbreviations("Color"))

In [43]:
data_frame.show(10)

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+------------------+--------------+---------------------+-----------+---------+---------+-----------------+----------------------+----------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|          Location|Violation code|Violation Description|Fine amount| Latitude|Longitude|Color Description|Body Style Description|color long|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+------------------+--------------+---------------------+-----------+---------+---------+-----------------+----------------------+----------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|   GY|   13147 WELBY WAY|        4000A1|   NO EVIDENCE OF REG|       50

In [44]:
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: float (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)
 |-- color long: string (nullable = true)



In [45]:
data_frame.select('Color','color long').show()

+-----+----------+
|Color|color long|
+-----+----------+
|   GY|      Gray|
|   WH|     White|
|   BK|     Black|
|   WH|     White|
|   BK|     Black|
|   GY|      Gray|
|   BL|      Blue|
|   BK|     Black|
|   BR|     Brown|
|   SI|    Silver|
|   WH|     White|
|   GO|      Gold|
|   BK|     Black|
|   BK|     Black|
|   BK|     Black|
|   BK|     Black|
|   WH|     White|
| null|      null|
|   BK|     Black|
|   BK|     Black|
+-----+----------+
only showing top 20 rows



In [46]:
# Most frequent colour values for Hondas
data_frame = data_frame.select("color long", f.when(data_frame.Make == "HOND",1))\
   .groupBy('color long')\
    .count()\
    .sort('count', ascending=False)\
    .show(10)

+----------+-------+
|color long|  count|
+----------+-------+
|     White|2180048|
|     Black|2131952|
|      Gray|1925995|
|    Silver|1288681|
|      Blue| 767381|
|       Red| 519774|
|     Green| 275246|
|     Brown| 199532|
|      Gold| 127213|
|    Maroon| 117267|
+----------+-------+
only showing top 10 rows

